In [17]:
import datetime
from pyspark.sql import SparkSession
def map_char(line):
    for x in line:
        return (x, 1)

def GC_content_count(path, filename):
    for i in range(0,4):
        cores = 2**i
        print(cores)
        spark_session = SparkSession\
            .builder\
            .master("spark://192.168.1.153:7077") \
            .appName("team8")\
            .config("spark.dynamicAllocation.enabled", True)\
            .config("spark.shuffle.service.enabled", True)\
            .config("spark.dynamicAllocation.executorIdleTimeout","300s")\
            .config("spark.cores.max", cores)\
            .getOrCreate()
        sc = spark_session.sparkContext
        start = datetime.datetime.now()
        microbiome = sc.textFile(path)
        microbiome_lines_indices = microbiome.zipWithIndex().map(lambda x: (x[1],x[0]))
        microbiome_characters = microbiome_lines_indices.filter(lambda x: x[0]%4 == 1).map(lambda x: list(x[1]))
        microbiome_mapped_reduced = microbiome_characters.map(map_char).reduceByKey(lambda x, y: x + y)
        total_sum = microbiome_mapped_reduced.map(lambda x: x[1]).sum()
        g = microbiome_mapped_reduced.filter(lambda x: x[0] == "G").take(1)[0][1]
        c = microbiome_mapped_reduced.filter(lambda x: x[0] == "C").take(1)[0][1]
        GC_percentage = ((g+c)/total_sum)*100
        end = datetime.datetime.now()
        total_time = end - start
        print(f'GC content in the file "{filename}" is {GC_percentage:0.2f}%')
        print(f'Total runtime: {total_time}')
        sc.stop()
    print("End")

In [18]:
GC_content_count("hdfs://192.168.1.153:9000/team08/ten_point_six_gb.tar.bz2", "ten_point_six_gb.tar.bz2")

1


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.NullPointerException
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:560)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:238)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.GatewayConnection.run(GatewayConnectio

In [ ]:
GC_content_count("hdfs://192.168.1.153:9000/team08/SRS143728.tar.bz2", "SRS143728.tar.bz2")

In [ ]:
#.config("spark.cores.max", cores)\
#.config("spark.executor.cores", 4)\